In [2]:
!pip install ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.9/722.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [3]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import warnings
warnings.simplefilter("ignore")
from ultralytics import YOLO
from google.colab.patches import cv2_imshow

In [5]:
def significant_pose_detection(arr, th):
  n = len(arr)
  i=0
  for itemm in arr:
    if np.sum(itemm[0]+itemm[1])!=0:
      i = i + 1

  sig_det = (i/n)
  if sig_det>=th:
    return 1

  return 0


def merge_image(back, front, x,y):
    if back.shape[2] == 3:
        back = cv2.cvtColor(back, cv2.COLOR_BGR2BGRA)
    if front.shape[2] == 3:
        front = cv2.cvtColor(front, cv2.COLOR_BGR2BGRA)

    bh,bw = back.shape[:2]
    fh,fw = front.shape[:2]
    x1, x2 = max(x, 0), min(x+fw, bw)
    y1, y2 = max(y, 0), min(y+fh, bh)
    front_cropped = front[y1-y:y2-y, x1-x:x2-x]
    back_cropped = back[y1:y2, x1:x2]

    alpha_front = front_cropped[:,:,3:4] / 255
    alpha_back = back_cropped[:,:,3:4] / 255

    result = back.copy()
    print(f'af: {alpha_front.shape}\nab: {alpha_back.shape}\nfront_cropped: {front_cropped.shape}\nback_cropped: {back_cropped.shape}')
    result[y1:y2, x1:x2, :3] = alpha_front * front_cropped[:,:,:3] + (1-alpha_front) * back_cropped[:,:,:3]
    result[y1:y2, x1:x2, 3:4] = (alpha_front + alpha_back) / (1 + alpha_front*alpha_back) * 255

    return result


model = YOLO('yolov8n-pose.pt')

image_file_name = "Q3_pose.png"
img = cv2.imread(image_file_name)
results = model.predict(image_file_name)[0]
th = 0.3

Q = []
det = 0
concat_df = pd.DataFrame()

for result in results:
  landmarks = []
  kpts = result.keypoints
  nk = kpts.shape[1]

  for i in range(nk):
    keypoint = kpts.xy[0, i]
    x, y = int(keypoint[0].item()), int(keypoint[1].item())
    landmarks.append([x,y])

  detections = significant_pose_detection(landmarks, th)
  if detections==1:
    det = det + 1
    for j in range(len(landmarks)):
      x = landmarks[j][0]
      y = landmarks[j][1]
      cv2.circle(img, (x, y), 3, (0, 0, 255), -1)
      ttext = "("+str(x)+","+str(y)+")"
      cv2.putText(img, ttext, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
                    0.3, (0,0,0), 1, cv2.LINE_AA)

    df = pd.DataFrame(landmarks, columns =['x'+str(det), 'y'+str(det)], dtype = float)
    concat_df = pd.concat([concat_df, df], axis=1)

    cv2.line(img, landmarks[11], landmarks[12], (0,255,0), 2)
    a = (landmarks[11][0] + landmarks[12][0])//2
    b = (landmarks[11][1] + landmarks[12][1])//2
    #print(a," ",b)
    #cv2.circle(img, (a, b), 5, (0, 255, 255), -1)
    #cv2.circle(img, (a, b), 7, (0, 0, 255), 2)
    Q.append([a,b])

# m = list(map(lambda l: [v for v in l if v != 0], sorted(Q)))
m = list(map(lambda l: [v for v in l if v != 0], Q))
nQ = [ele for ele in m if ele != []]

#k, _, _ = results.keypoints.xy.cpu().numpy().shape
k = len(nQ)

i=0
while i<(k-1):
  cv2.line(img, nQ[i], nQ[i+1], (255, 0,0), 2)
  i = i + 1

i = 0
while i<k:
  cv2.circle(img, nQ[i], 5, (0, 255, 255), -1)
  cv2.circle(img, nQ[i], 7, (0, 0, 255), 2)
  i = i + 1

concat_df.to_csv('detected_pose.csv')
df_nQ = pd.DataFrame(nQ, columns =['x', 'y'], dtype = int)
df_nQ.to_csv('nQ_centroids.csv')

print("\n", k, "Persons detected.")
cv2.imwrite("output.jpg", img)
# cv2_imshow(img)
h,w,c = img.shape

canva = np.zeros([h,w,3],dtype=np.uint8)  # change the h, w

nQT = [[itemm[1],itemm[0]] for itemm in nQ]

i=0
while i<(k-1):
  cv2.line(canva, nQ[i], nQ[i+1], (255, 0,0), 2)
  i = i + 1

i = 0
while i<k:
  cv2.circle(canva, nQ[i], 5, (0, 255, 255), -1)
  cv2.circle(canva, nQ[i], 7, (0, 0, 255), 2)
  ttext = "("+str(nQ[i][0])+","+str(nQ[i][1])+")"

  i = i + 1

canva = cv2.rotate(canva, cv2.ROTATE_90_CLOCKWISE)

i = 0

while i<k:
  ttext = "("+str(nQ[i][0])+","+str(nQ[i][1])+")"
  cv2.putText(canva, ttext, (h-nQT[i][0]+15, nQT[i][1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,255,255), 1, cv2.LINE_AA)
  cv2.putText(canva, "Person_"+str(i+1), (h-nQT[i][0]+15, nQT[i][1]), cv2.FONT_HERSHEY_SIMPLEX,0.3, (0,255,0), 1, cv2.LINE_AA)
  i = i + 1
overlay = canva.copy()

x, y, w1, h1 = h-nQT[0][0]-25, nQT[0][1]-20, 50, nQT[k-1][1]-nQT[0][1]+40
cv2.rectangle(overlay, (x, y), (x+w1, y+h1), (0, 200, 0), -1)

alpha = 0.3

canva_new = cv2.addWeighted(overlay, alpha, canva, 1 - alpha, 0)

h, w, _ = canva_new.shape
img_resize = cv2.resize(img, (300,200))
canva = merge_image(canva_new, img_resize, w-300,h-200)

cv2.imwrite("canvas_orig.jpg", canva)
# cv2_imshow(canva)


image 1/1 /content/Q3_pose.png: 448x640 12 persons, 304.0ms
Speed: 19.3ms preprocess, 304.0ms inference, 24.9ms postprocess per image at shape (1, 3, 448, 640)

 8 Persons detected.
af: (200, 300, 1)
ab: (200, 300, 1)
front_cropped: (200, 300, 4)
back_cropped: (200, 300, 4)


True

In [6]:
url = 'nQ_centroids.csv'
data = pd.read_csv(url)
x = data['x'].values
y = data['y'].values

slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

y_pred = intercept + slope * x
residuals = y - y_pred

residuals_std = np.std(residuals)

ci_lower = y_pred - 1.440*residuals_std
ci_upper = y_pred + 1.440*residuals_std

outliers = np.where((y < ci_lower) | (y > ci_upper))

print("Outliers: ", outliers)

img = cv2.imread('Q3_pose.png')

persons = concat_df.values.reshape(-1, 8, 2)

# for i in outliers[0]:
#     x_coords = concat_df['x'+str(i+1)].dropna().values
#     y_coords = concat_df['y'+str(i+1)].dropna().values
#     person = np.column_stack((x_coords, y_coords))
#     person = person[~np.all(person == [0, 0], axis=1)]
#     pts = np.array(person, np.int32)
#     print(pts)
#     cv2.polylines(img, [pts], True, (0,0,255), 2)

# cv2.imwrite('outliers.png', img)

for i in outliers[0]:
    x_coords = concat_df['x'+str(i+1)].dropna().values
    y_coords = concat_df['y'+str(i+1)].dropna().values
    person = np.column_stack((x_coords, y_coords))
    person = person[~np.all(person == [0, 0], axis=1)]
    x_min, y_min = np.min(person, axis=0)
    x_max, y_max = np.max(person, axis=0)
    cv2.rectangle(img, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0,0,255), 2)

cv2.imwrite('outliers.png', img)

Outliers:  (array([0, 7]),)


True